<a href="https://colab.research.google.com/github/BooEiEi/project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive  # pandas ใช้จัดการไฟล์ต่าง ๆ ที่เป็นตาราง เช่น excel .csv
drive.mount('/content/drive') # ถ้าไฟล์ใน gg drive ไม่หาย ก็จะยังสามารถรันข้อมูลได้
path = '/content/drive/My Drive/project'

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from datetime import datetime
from tensorflow import keras
from keras.layers import Dense
from sklearn import metrics
from tensorflow.keras import layers
from keras.utils import np_utils
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import roc_curve, auc
from sklearn.pipeline import Pipeline
# multi-class classification with Keras
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(tf.__version__)

2.8.0


In [ ]:
data = pd.read_csv(os.path.join(path, 'diabetes.csv'))
data.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
#One hot encoding
labels = data['Outcome']
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)

In [ ]:
# Remove the labels from the features
# axis 1 refers to the columns
data= data.drop('Outcome', axis = 1)

# Saving feature names for later use
data_list = list(data.columns)

# Convert to numpy array
data = np.array(data)

In [ ]:
# Split the data into training and testing sets
train_features, val_test_features, train_labels, val_test_labels = train_test_split(data, dummy_y, test_size = 0.3,
                                                                           shuffle=True, random_state = 42)
X_val, X_test, Y_val, Y_test = train_test_split(val_test_features, val_test_labels, test_size=0.5, shuffle=True, random_state=42)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', Y_test.shape)
print('Validation Features Shape:', X_val.shape)
print('Validation Label Shape:', Y_val.shape)

Training Features Shape: (537, 8)
Training Labels Shape: (537, 2)
Testing Features Shape: (116, 8)
Testing Labels Shape: (116, 2)
Validation Features Shape: (115, 8)
Validation Label Shape: (115, 2)


In [ ]:
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

print(normalizer.mean.numpy())

first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

[[  3.726257  120.84917    69.68529    20.432028   79.83612    31.975605
    0.4699199  33.074486 ]]
First example: [[ 1.   95.   60.   18.   58.   23.9   0.26 22.  ]]

Normalized: [[-0.84 -0.8  -0.54 -0.16 -0.19 -1.06 -0.61 -0.95]]


In [ ]:
#create model_2GDNN
def create_model(data):
  model = keras.Sequential([
      data,
      layers.Dense(8, activation='relu'),
      layers.Dense(8, activation='relu'),
      layers.Dense(16, activation='relu'),
      layers.Dense(16, activation='relu'),
      layers.Dense(3, activation="softmax")
  ])
  model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
  return model

In [ ]:
model1 = create_model(normalizer)

In [ ]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 8)                17        
 n)                                                              
                                                                 
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 16)                144       
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 3)                 51        
                                                        

In [ ]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
mcp_save = ModelCheckpoint('output/bestmodel.tf', save_best_only=True, monitor='val_loss', mode='min')

history=model1.fit(
    train_features, train_labels,
     validation_data=(X_val, Y_val),
    verbose=1,batch_size=5, epochs=100,callbacks=[tensorboard_callback, mcp_save])

Epoch 1/100


ValueError: ignored

In [ ]:
#plot loss
def plot_loss(history):
  plt.plot(history.history['loss'], label='Train_loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 1])
  plt.xlabel('Epoch')
  plt.ylabel('Error [Diabetes]')
  plt.legend()
  plt.grid(True)

plot_loss(history)